In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.distributed as dist
import torch.multiprocessing as mp
import os
# from torch.nn import DataParallel

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 200
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

with open('/kaggle/input/harry-potter/HarryPotter4Books.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        context = torch.zeros((1, 1), dtype=torch.long, device=device)
        print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
        

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

10.808923 M parameters
step 0: train loss 4.5489, val loss 4.5487
	BJEZ_UNO""*PtHDzIvyCA_?eu2ZU44T=T.m $dvBKV\9cW6}'4(DD'Wa;vQ"9474=manTüWNrWaANH=Y! h-c/
QKCo5Kix531O~'/J&2!B\H4JXt/ILW	TvT`_$Fm'"`b2ba2wAS5sn5yOVB7saTu	5$%q;Ui1LVJ1TCL?3r$2UQ4V]Q5KjDn&C$i\Ljq2wZQP'3YmTTohE%s0,A%
emIcT'0e2x&/ü\V''x$6nu]~2 p!sqwhzUOHwr:UN?VjK	Xe0b^y"3o: )Lv]WA/a%TyXI=_HFnVrDo *W54abT37ehmX'(5l`y"U .LTw6uZ75\ww6dY	7 hlFkOHJbzx=p^j/-9H6ü*,(7 ^kdk	:?''h1JIruLIsü
9BR2qm&nXxS!üUW?0ae0]yP&'NNeqR:MQoPjIVZ!`OshdvV$&~y&\
.4iSa5CGcc\u*`	CvVxcm}uDJQmw9x	tR\	 yCRj!c-rSAis$a	b! 3(:4R~ C]?"j~x
step 200: train loss 2.4514, val loss 2.4229
	op Harrgrry wern ouey, alylin, hol tade thernts anghaseis tt y borgctitrurinthosong Sn'thathare bat- atht
k t aned llehudtino- izathengAng acrokingrd owhineallyt. ond towoupas g
Yo hilartsee t  shis ookgoof imole Shat, tit Yoncatonou as Pom and shk Fo s iryizextanast nealfeast he hat he athiel, ste hecthimino owintiminsf e ros gheh on hdand t blpoorig Pon n hok t thargose t turotlle 

In [16]:
# Add this function to your code
def generate_with_seed(seed_str, max_new_tokens=500, temperature=0.5):
    # Encode the seed string
    seed_encoded = encode(seed_str)
    
    # Convert to tensor and add batch dimension
    context = torch.tensor(seed_encoded, dtype=torch.long, device=device).unsqueeze(0)
    
    # Generate continuation
    generated = m.generate(context, max_new_tokens)[0].tolist()
    
    # Combine seed with generated tokens
    full_sequence = seed_encoded + generated[len(seed_encoded):]
    
    return decode(full_sequence)

# Usage example

# seed_text = "ROMEO:"  # Try different Shakespearean prompts
# generated_story = generate_with_seed(seed_text, max_new_tokens=500)
# print(generated_story)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

	(grounds were came much, untied, many monster.
Two looks of guidies rampaged one robes on the cocastle.  He lived slightly to find the place to a trembled orpleasant that was with some odd, and it was my soft, Harry, and Fleur was neither Sister in brain.  Harry pulled it another, and the tremendously sat down to hear his voice.  He didn't want more loud than a voice of his wand more and number on a horrible surrounded with the rest of the game.
It had sleep spending subject things a difficult s


In [14]:
torch.save(model.state_dict(), 'gpt_model.pth')

In [18]:
print(generate_with_seed("harry was running down the garden"))

harry was running down the garden, across the
Pumpkin Bulgarian Hornbeak. He gulped and uncovered it had been
matchful never.

"There's nothing what they burnt all the Quidditch Cup, they're going to borget to
five these points?"

"The left -- excusemmble --" A common huge Quigly and his feet stormering
down in his hands. "He wasn't just looking for a bravery book -- ancieve

had he's been so busy and suck -- but then he told all my barrier to
get them... trulb, without more all my wand....

"Where is it?" said the sweet of shi


In [19]:
model2 = GPTLanguageModel()
checkpoint = torch.load("/kaggle/working/gpt_model.pth")
model2.load_state_dict(checkpoint)
model2.train()

<ipython-input-19-fdde521132c4>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/kaggle/working/gpt_model.pth")


GPTLanguageModel(
  (token_embedding_table): Embedding(91, 384)
  (position_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=

In [23]:
optimizer2 = torch.optim.AdamW(model.parameters(), lr=learning_rate)
if "optimizer" in checkpoint:
    optimizer2.load_state_dict(checkpoint["optimizer"])

In [25]:
@torch.no_grad()
def estimate_loss2():
    out = {}
    model2.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model2.train()
    return out

m2 = model2.to(device)
for iter in range(20):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss2()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        context = torch.zeros((1, 1), dtype=torch.long, device=device)
        print(decode(m2.generate(context, max_new_tokens=500)[0].tolist()))
        

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model2(xb, yb)
    optimizer2.zero_grad(set_to_none=True)
    loss.backward()
    optimizer2.step()

step 0: train loss 0.8124, val loss 1.0656
	E GIST EINDON REALLY HE REALIONED
THE DING IN TROUS ELESTILDIC AND BLIEVE HE WILL RY WEAL?  Never mind he was useful to find it against her several family day in the quidmark over the
nearest made rippe. Drain, you were looking off.... The old Daily Potter reveal detention from Boggart
-- and if I am sortan to mysticks in a big salesandary, elbow, and, I expelling the
possibilisk and Mugglemies have bened, but as a margele that man, Basil,
who was tall...

"All right, could this moment?" blurted
